In [109]:
import requests
import pandas as pd

In [110]:
# this function will accept a FIADB-API fullreport URL and return dataframes for the estimates as well as subtotals, and totals where available.
def fiadb_api_GET(url):
    # make request
    resp = requests.get(url)
    # parse response to json
    data = resp.json()

    # create output dictionary and populate it with estimate data frames
    outDict = {}
    # append estimates
    outDict['estimates'] = pd.DataFrame(data['estimates'])

    # append subtotals and totals if present
    if 'subtotals' in data.keys():
        subT = {}
        for i in data['subtotals'].keys():
            subT[i] = pd.DataFrame(data['subtotals'][i])
        outDict['subtotals'] = subT
        outDict['totals'] = pd.DataFrame(data['totals'])

    # append metadata
    outDict['metadata'] = data['metadata']
    return outDict

In [111]:
# example of usage
url = "https://apps.fs.usda.gov/fiadb-api/fullreport?rselected=County%20code%20and%20name&cselected=Ownership%20class&schema=FS_FIADB.&snum=2&wc=342021&outputFormat=NJSON"
getData = fiadb_api_GET(url=url)

In [112]:
# estimate data frame
df = getData['estimates']


In [113]:
df.head()

,ESTIMATE,GRP1,GRP2,PLOT_COUNT,SE,SE_PERCENT,VARIANCE
0,506.868805,`1773 34001 NJ Atlantic,`0004 Fish and Wildlife Service,1,506.868805,100.000000,2.569160e+05
1,3570.939143,`1773 34001 NJ Atlantic,`0006 Other federal,1,3593.243520,100.624608,1.291140e+07
2,59465.501250,`1773 34001 NJ Atlantic,`0007 State,21,12981.976202,21.831105,1.685317e+08
3,46432.619674,`1773 34001 NJ Atlantic,`0008 County and Municipal,13,12366.672551,26.633588,1.529346e+08
4,73625.650784,`1773 34001 NJ Atlantic,`0010 Private,25,14630.928291,19.872053,2.140641e+08


In [114]:
df = df.drop(columns = ["PLOT_COUNT", "SE", "SE_PERCENT", "VARIANCE"], axis = 1)

In [115]:
df["GRP2"] = df["GRP2"].str.split(" ", n = 1).str[1] 

df["GRP1"] = df["GRP1"].str.split(" ", n = 1).str[1]

In [116]:
df.head()

,ESTIMATE,GRP1,GRP2
0,506.868805,34001 NJ Atlantic,Fish and Wildlife Service
1,3570.939143,34001 NJ Atlantic,Other federal
2,59465.501250,34001 NJ Atlantic,State
3,46432.619674,34001 NJ Atlantic,County and Municipal
4,73625.650784,34001 NJ Atlantic,Private


In [117]:
#County code and name 	Total 	National Park Service 	Fish and Wildlife Service 	Dept of Defense 	Other federal 	State 	County and Municipal 	Other local govt 	Private

#return products.melt(id_vars = "product_id", value_vars = ["store1","store2","store3"],  var_name = "store", value_name = "price").dropna()


df = df.pivot(index = "GRP1", columns = "GRP2", values = "ESTIMATE")




In [118]:

df.head()

GRP2,County and Municipal,Dept of Defense,Fish and Wildlife Service,National Park Service,Other federal,Other local govt,Private,State
GRP1,,,,,,,,
34001 NJ Atlantic,46432.619674,NaN,506.868805,NaN,3570.939143,NaN,73625.650784,59465.501250
34003 NJ Bergen,3233.604712,NaN,NaN,NaN,NaN,NaN,4679.383252,9931.613594
34005 NJ Burlington,10965.412506,19626.162081,NaN,NaN,NaN,NaN,105640.324309,143446.556135
34007 NJ Camden,20728.499223,NaN,NaN,NaN,NaN,NaN,10105.600176,10918.013029
34009 NJ Cape May,300.403451,NaN,13283.906769,NaN,NaN,NaN,16646.761715,30521.426453


In [119]:
df = df.reset_index()

In [120]:
#df.columns = df.columns.to_flat_index()

df = df.rename(columns = {"GRP1" : "County code and name"})



In [121]:
df.head()

GRP2,County code and name,County and Municipal,Dept of Defense,Fish and Wildlife Service,National Park Service,Other federal,Other local govt,Private,State
0,34001 NJ Atlantic,46432.619674,NaN,506.868805,NaN,3570.939143,NaN,73625.650784,59465.501250
1,34003 NJ Bergen,3233.604712,NaN,NaN,NaN,NaN,NaN,4679.383252,9931.613594
2,34005 NJ Burlington,10965.412506,19626.162081,NaN,NaN,NaN,NaN,105640.324309,143446.556135
3,34007 NJ Camden,20728.499223,NaN,NaN,NaN,NaN,NaN,10105.600176,10918.013029
4,34009 NJ Cape May,300.403451,NaN,13283.906769,NaN,NaN,NaN,16646.761715,30521.426453


In [122]:
df.index = df.index.rename("")
df.columns = [''.join(str(s) for s in col if s) for col in df.columns]

In [123]:
#round numbers
#add total row
#add total column
#change NaN to -
#Change grp1 = county code and name

df.head()

,County code and name,County and Municipal,Dept of Defense,Fish and Wildlife Service,National Park Service,Other federal,Other local govt,Private,State
,,,,,,,,,
0,34001 NJ Atlantic,46432.619674,NaN,506.868805,NaN,3570.939143,NaN,73625.650784,59465.501250
1,34003 NJ Bergen,3233.604712,NaN,NaN,NaN,NaN,NaN,4679.383252,9931.613594
2,34005 NJ Burlington,10965.412506,19626.162081,NaN,NaN,NaN,NaN,105640.324309,143446.556135
3,34007 NJ Camden,20728.499223,NaN,NaN,NaN,NaN,NaN,10105.600176,10918.013029
4,34009 NJ Cape May,300.403451,NaN,13283.906769,NaN,NaN,NaN,16646.761715,30521.426453


In [124]:
df = df[["County code and name","National Park Service","Fish and Wildlife Service","Dept of Defense","Other federal","State","County and Municipal","Other local govt","Private"]]

In [125]:
df.head()

,County code and name,National Park Service,Fish and Wildlife Service,Dept of Defense,Other federal,State,County and Municipal,Other local govt,Private
,,,,,,,,,
0,34001 NJ Atlantic,NaN,506.868805,NaN,3570.939143,59465.501250,46432.619674,NaN,73625.650784
1,34003 NJ Bergen,NaN,NaN,NaN,NaN,9931.613594,3233.604712,NaN,4679.383252
2,34005 NJ Burlington,NaN,NaN,19626.162081,NaN,143446.556135,10965.412506,NaN,105640.324309
3,34007 NJ Camden,NaN,NaN,NaN,NaN,10918.013029,20728.499223,NaN,10105.600176
4,34009 NJ Cape May,NaN,13283.906769,NaN,NaN,30521.426453,300.403451,NaN,16646.761715


In [129]:
df = df.style.hide(axis = "index")

In [133]:
df

County code and name,National Park Service,Fish and Wildlife Service,Dept of Defense,Other federal,State,County and Municipal,Other local govt,Private
34001 NJ Atlantic,nan,506.868805,nan,3570.939143,59465.501250,46432.619674,nan,73625.650784
34003 NJ Bergen,nan,nan,nan,nan,9931.613594,3233.604712,nan,4679.383252
34005 NJ Burlington,nan,nan,19626.162081,nan,143446.556135,10965.412506,nan,105640.324309
34007 NJ Camden,nan,nan,nan,nan,10918.013029,20728.499223,nan,10105.600176
34009 NJ Cape May,nan,13283.906769,nan,nan,30521.426453,300.403451,nan,16646.761715
34011 NJ Cumberland,nan,nan,nan,nan,66973.090495,8516.423980,nan,73311.952143
34013 NJ Essex,nan,nan,nan,nan,4348.589171,3622.860777,nan,nan
34015 NJ Gloucester,nan,nan,nan,3177.762804,2856.194620,5544.611276,nan,49994.386300
34017 NJ Hudson,nan,nan,nan,nan,1517.090791,nan,nan,nan
34019 NJ Hunterdon,nan,nan,nan,nan,9898.998646,9505.545680,nan,66653.027001
